# Train a Scikit-Learn model in SageMaker and track with MLFlow

## Setup Environment

In [2]:
!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.10 requires mkl, which is not installed.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
jupyter-server 2.14.2 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.


In [9]:
!pip install --upgrade numpy scikit-learn

  Using cached numpy-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached numpy-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
Using cached scikit_learn-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.10 requires mkl, which is not installed.
hdijupyterutils 0.21.0 requires panda

In [1]:
import sagemaker
import pandas as pd
# from sklearn.datasets import load_boston
from sagemaker.sklearn.estimator import SKLearn
# from sklearn.model_selection import train_test_split

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()

# uri of your remote mlflow server
tracking_uri = 'http://MLflow-MLFLO-mqNDqavgvj1j-9a99f36f0a9351c0.elb.us-east-1.amazonaws.com/' 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Prepare data
We load a dataset from sklearn, split it and send it to S3

In [5]:
# we use the Boston housing dataset 
# data = load_boston()

# X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.25, random_state=42)

# trainX = pd.DataFrame(X_train, columns=data.feature_names)
# trainX['target'] = y_train

# testX = pd.DataFrame(X_test, columns=data.feature_names)
# testX['target'] = y_test

# trainX.to_csv('boston_train.csv')
# testX.to_csv('boston_test.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [2]:
# send data to S3. SageMaker will take training data from s3
train_path = sess.upload_data(path='boston_train.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')
test_path = sess.upload_data(path='boston_test.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')

## Train

In [3]:
hyperparameters = {
    'tracking_uri': tracking_uri,
    'experiment_name': 'boston-housing-mlops',
    'n-estimators': 150,
    'min-samples-leaf': 4,
    'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
    'target': 'target'
}

metric_definitions = [{'Name': 'median-AE', 'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}]

estimator = SKLearn(
    entry_point='train.py',
    source_dir='source_dir',
    role=role,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.0-1',
    base_job_name='mlflow',
)

In [4]:
estimator.fit({'train':train_path, 'test': test_path})

INFO:sagemaker:Creating training-job with name: mlflow-2024-08-24-12-44-47-303


2024-08-24 12:44:49 Starting - Starting the training job...
2024-08-24 12:45:04 Starting - Preparing the instances for training...
2024-08-24 12:45:34 Downloading - Downloading input data...
2024-08-24 12:45:59 Downloading - Downloading the training image...
2024-08-24 12:46:45 Training - Training image download completed. Training in progress..2024-08-24 12:46:51,182 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-08-24 12:46:51,184 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-24 12:46:51,187 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-24 12:46:51,201 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-08-24 12:46:51,425 sagemaker-training-toolkit INFO     Installing module with the following command:
/miniconda3/bin/python -m pip install . -r requirements.txt
Processing /opt/ml/code
  Preparing metadata (setup.py):

      Successfully uninstalled pyarrow-14.0.1
2024-08-24 12:47:07,434 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-24 12:47:07,438 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-24 12:47:07,456 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-24 12:47:07,459 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-24 12:47:07,477 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-24 12:47:07,480 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-24 12:47:07,494 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
  


2024-08-24 12:47:20 Uploading - Uploading generated training model2024-08-24 12:47:12,336 sagemaker-containers INFO     Reporting training SUCCESS

2024-08-24 12:47:28 Completed - Training job completed
Training seconds: 115
Billable seconds: 115
